In [43]:
from fim import apriori, eclat, fpgrowth, fim
import numpy as np
import pandas as pd

In [2]:
ls

README.md                      mnist_8x8_image.csv
Untitled.ipynb                 mnist_8x8_label.csv
association_rules.ipynb        mnist_8x8_label_onehot.csv
association_rules.py           onigiri.csv
association_rules_julia.ipynb  store_data.csv
association_rules_pyfim.ipynb  store_data_trans.csv
mlxtend_mnist.ipynb


In [3]:
mnist_array = np.loadtxt("mnist_8x8_image.csv", delimiter=',')
mnist_array = mnist_array.astype(np.int64)
col_names = [str(i) for i in (range(mnist_array.shape[1]))]
mnist_df = pd.DataFrame(mnist_array, columns=col_names)
mnist_df.head()

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
1,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,1,1,0,0
2,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,1,1,1,0
3,0,0,0,1,1,0,0,0,0,1,...,1,0,0,0,0,1,1,1,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [4]:
mnist_label = np.loadtxt("mnist_8x8_label_onehot.csv", delimiter=",")
col_names = ['label_'+str(i) for i in range(mnist_label.shape[1])]
mnist_label = pd.DataFrame(mnist_label, columns=col_names)

mnist_label.head()

,label_0,label_1,label_2,label_3,label_4,label_5,label_6,label_7,label_8,label_9
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [40]:
mnist_label_int = np.argmax(mnist_label.values, axis=1)
mnist_label_int

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1,
       2, 3, 4, 5, 6, 7, 8, 9, 0, 9, 5, 5, 6, 5, 0, 9, 8, 9, 8, 4, 1, 7,
       7, 3, 5, 1, 0, 0, 2, 2, 7, 8, 2, 0, 1, 2, 6, 3, 3, 7, 3, 3, 4, 6,
       6, 6, 4, 9, 1, 5, 0, 9, 5, 2, 8, 2, 0, 0, 1, 7, 6, 3, 2, 1, 7, 4,
       6, 3, 1, 3, 9, 1, 7, 6, 8, 4, 3, 1, 4, 0, 5, 3, 6, 9, 6, 1, 7, 5,
       4, 4, 7, 2, 8, 2, 2, 5, 7, 9, 5, 4, 8, 8, 4, 9, 0, 8, 9, 8, 0, 1,
       2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3,
       4, 5, 6, 7, 8, 9, 0, 9, 5, 5, 6, 5, 0, 9, 8, 9, 8, 4, 1, 7, 7, 3,
       5, 1, 0, 0, 2, 2, 7, 8, 2, 0, 1, 2, 6, 3, 3, 7, 3, 3, 4, 6, 6, 6,
       4, 9, 1, 5, 0, 9, 5, 2, 8, 2, 0, 0, 1, 7, 6, 3, 2, 1, 7, 3, 1, 3,
       9, 1, 7, 6, 8, 4, 3, 1, 4, 0, 5, 3, 6, 9, 6, 1, 7, 5, 4, 4, 7, 2,
       8, 2, 2, 5, 5, 4, 8, 8, 4, 9, 0, 8, 9, 8, 0, 1, 2, 3, 4, 5, 6, 7,
       8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
       0, 9, 5, 5, 6, 5, 0, 9, 8, 9, 8, 4, 1, 7, 7,

In [5]:
def pickupColumns(df, ind):
    colnames = df.columns
    return  list(np.array(colnames[df.iloc[ind,:] ==1],dtype=np.int64))

In [42]:
transactions = [pickupColumns(mnist_df,i) for i in range(mnist_df.shape[0])]
transactions = np.array(transactions)

%time res = np.array(fpgrowth(transactions, supp=5, zmin=10, report='s'))

freq_df = pd.DataFrame(res, columns=['itemsets', 'support'])
freq_df.sort_values('support', ascending=False)

CPU times: user 1.3 s, sys: 51.5 ms, total: 1.35 s
Wall time: 1.36 s


,itemsets,support
3425,"(45, 53, 18, 10, 12, 60, 3, 4, 11, 59)",0.169265
344158,"(42, 34, 50, 53, 26, 18, 60, 3, 59, 11)",0.164811
342686,"(42, 34, 50, 26, 18, 51, 60, 3, 59, 11)",0.164811
24,"(18, 52, 10, 51, 12, 60, 3, 4, 11, 59)",0.164811
8225,"(13, 45, 53, 10, 12, 60, 3, 4, 11, 59)",0.163697
3874,"(45, 53, 26, 18, 10, 60, 3, 4, 59, 11)",0.16147
1196,"(53, 18, 10, 51, 12, 60, 3, 4, 11, 59)",0.16147
46451,"(50, 18, 10, 51, 12, 60, 3, 4, 11, 59)",0.16147
51020,"(50, 45, 53, 18, 10, 60, 3, 4, 11, 59)",0.16147
1140,"(53, 52, 10, 51, 12, 60, 3, 4, 11, 59)",0.158129


In [63]:
transactions = [pickupColumns(mnist_df,i) for i in range(mnist_df.shape[0])]
transactions = np.array(transactions)

%time res = np.array(apriori(transactions, supp=5, zmin=10, report='s'))

freq_df = pd.DataFrame(res, columns=['itemsets', 'support'])
freq_df = freq_df.sort_values('support', ascending=False)
freq_df.index = np.arange(freq_df.shape[0])
freq_df.head()

CPU times: user 7.1 s, sys: 150 ms, total: 7.25 s
Wall time: 7.28 s


,itemsets,support
0,"(45, 53, 18, 10, 12, 60, 3, 4, 11, 59)",0.169265
1,"(42, 34, 50, 53, 26, 18, 60, 3, 59, 11)",0.164811
2,"(42, 34, 50, 26, 18, 51, 60, 3, 59, 11)",0.164811
3,"(18, 52, 10, 51, 12, 60, 3, 4, 11, 59)",0.164811
4,"(13, 45, 53, 10, 12, 60, 3, 4, 11, 59)",0.163697


In [60]:
transactions = [pickupColumns(mnist_df,i) for i in range(mnist_df.shape[0])]
transactions = np.array(transactions)

%time res = np.array(eclat(transactions, supp=5, zmin=10, report='s'))

freq_df = pd.DataFrame(res, columns=['itemsets', 'support'])
freq_df = freq_df.sort_values('support', ascending=False)
freq_df.index = np.arange(freq_df.shape[0])
freq_df.head()

CPU times: user 1.7 s, sys: 13.6 ms, total: 1.71 s
Wall time: 1.72 s


,itemsets,support
0,"(45, 10, 12, 18, 4, 53, 60, 11, 3, 59)",0.169265
1,"(42, 26, 50, 51, 3, 18, 34, 59, 60, 11)",0.164811
2,"(42, 53, 26, 50, 34, 3, 11, 59, 60, 18)",0.164811
3,"(18, 52, 10, 51, 60, 4, 12, 3, 11, 59)",0.164811
4,"(13, 45, 10, 12, 53, 60, 11, 4, 3, 59)",0.163697


In [61]:
transactions = [pickupColumns(mnist_df,i) for i in range(mnist_df.shape[0])]
transactions = np.array(transactions)

%time res = np.array(fpgrowth(transactions, supp=5, zmin=10, report='s'))

freq_df = pd.DataFrame(res, columns=['itemsets', 'support'])
freq_df = freq_df.sort_values('support', ascending=False)
freq_df.index = np.arange(freq_df.shape[0])
freq_df.head()

CPU times: user 1.42 s, sys: 72.6 ms, total: 1.49 s
Wall time: 1.52 s


,itemsets,support
0,"(45, 53, 18, 10, 12, 60, 3, 4, 11, 59)",0.169265
1,"(42, 34, 50, 53, 26, 18, 60, 3, 59, 11)",0.164811
2,"(42, 34, 50, 26, 18, 51, 60, 3, 59, 11)",0.164811
3,"(18, 52, 10, 51, 12, 60, 3, 4, 11, 59)",0.164811
4,"(13, 45, 53, 10, 12, 60, 3, 4, 11, 59)",0.163697


In [62]:
transactions = [pickupColumns(mnist_df,i) for i in range(mnist_df.shape[0])]
transactions = np.array(transactions)

%time res = np.array(fim(transactions, supp=5, zmin=10, report='s'))

freq_df = pd.DataFrame(res, columns=['itemsets', 'support'])
freq_df = freq_df.sort_values('support', ascending=False)
freq_df.index = np.arange(freq_df.shape[0])
freq_df.head()

CPU times: user 1.52 s, sys: 97.7 ms, total: 1.62 s
Wall time: 1.69 s


,itemsets,support
0,"(45, 53, 18, 10, 12, 60, 3, 4, 11, 59)",0.169265
1,"(42, 34, 50, 53, 26, 18, 60, 3, 59, 11)",0.164811
2,"(42, 34, 50, 26, 18, 51, 60, 3, 59, 11)",0.164811
3,"(18, 52, 10, 51, 12, 60, 3, 4, 11, 59)",0.164811
4,"(13, 45, 53, 10, 12, 60, 3, 4, 11, 59)",0.163697


In [59]:
freq_df

,itemsets,support
0,"(45, 53, 18, 10, 12, 60, 3, 4, 11, 59)",0.169265
1,"(42, 34, 50, 53, 26, 18, 60, 3, 59, 11)",0.164811
2,"(42, 34, 50, 26, 18, 51, 60, 3, 59, 11)",0.164811
3,"(18, 52, 10, 51, 12, 60, 3, 4, 11, 59)",0.164811
4,"(13, 45, 53, 10, 12, 60, 3, 4, 11, 59)",0.163697
5,"(45, 53, 26, 18, 10, 60, 3, 4, 59, 11)",0.16147
6,"(53, 18, 10, 51, 12, 60, 3, 4, 11, 59)",0.16147
7,"(50, 18, 10, 51, 12, 60, 3, 4, 11, 59)",0.16147
8,"(50, 45, 53, 18, 10, 60, 3, 4, 11, 59)",0.16147
9,"(53, 52, 10, 51, 12, 60, 3, 4, 11, 59)",0.158129
